In [9]:
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
from PIL import Image
import tensorflow as tf 
from keras.applications.efficientnet import EfficientNetB0
from keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Dense
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import plotly.express as px
from keras.callbacks import ModelCheckpoint
import os


In [10]:
#descargar el modelo de face detection de yolov8
#model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")

In [11]:
data_files = {"train": ["./Dataset/train/data-00000-of-00004.arrow","./Dataset/train/data-00001-of-00004.arrow","./Dataset/train/data-00002-of-00004.arrow","./Dataset/train/data-00003-of-00004.arrow"]}
my_dataset = datasets.load_dataset("arrow", data_files=data_files, split="train")
df=my_dataset.to_pandas()
df.head()

,image,age,gender,race,service_test
0,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,6,0,0,True
1,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,4,1,1,False
2,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,1,1,2,False
3,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1,True
4,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,3,1,1,True


In [12]:
x=[]
y=[]
for item in df["image"]:
    image_bytes = item['bytes']
    image_array = np.frombuffer(image_bytes, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (224, 224))
    x.append(image)

y_age = df['age']
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(x, y_age, test_size=0.2, stratify=y_age)


In [13]:
detect_face_model = YOLO("C:\\Users\\ma907\\.cache\\huggingface\\hub\\models--arnabdhar--YOLOv8-Face-Detection\\snapshots\\52fa54977207fa4f021de949b515fb19dcab4488\\model.pt")
image_path = "morus.jpg"
output = detect_face_model(Image.open(image_path))
results = Detections.from_ultralytics(output[0])

print(results)


0: 384x640 1 FACE, 288.0ms
Speed: 19.5ms preprocess, 288.0ms inference, 21.8ms postprocess per image at shape (1, 3, 384, 640)
Detections(xyxy=array([[     530.77,      169.89,      735.34,      496.53]], dtype=float32), mask=None, confidence=array([    0.92215], dtype=float32), class_id=array([0]), tracker_id=None, data={'class_name': array(['FACE'], dtype='<U4')})


In [14]:
#cargar el modelo
detect_face_model = YOLO("C:\\Users\\ma907\\.cache\\huggingface\\hub\\models--arnabdhar--YOLOv8-Face-Detection\\snapshots\\52fa54977207fa4f021de949b515fb19dcab4488\\model.pt")
#modelo EfficientNetB0
base_model=EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224,224,3))


age_estimator_model=Sequential()
age_estimator_model.add(base_model)
age_estimator_model.add(GlobalAveragePooling2D())
age_estimator_model.add(BatchNormalization())
age_estimator_model.add(Dropout(0.2))
age_estimator_model.add(Dense(7, activation='linear'))

adam_optimizer = Adam(learning_rate=1e-4)
age_estimator_model.compile(optimizer=adam_optimizer, 
                     loss='mean_absolute_error', 
                     metrics=['mae', 'accuracy'])

def combined_model(image_path):
    # Cargar la imagen
    image = cv2.imread(image_path)
    
    output = detect_face_model(image)
    results= Detections.from_ultralytics(output[0])

    # Si se detecta al menos un rostro, recortar el rostro y pasar al modelo de edad
    if len(results.xyxy) > 0:
        box = results.xyxy[0]
        x1, y1, x2, y2 = box
        face = image[int(y1):int(y2), int(x1):int(x2)]
        face = cv2.resize(face, (224, 224))
        face = tf.expand_dims(face, axis=0)  # Agregar dimensión de lote
        return age_estimator_model(face)
    else:
        # Si no se detecta ningún rostro, devolver un valor predeterminado
        return tf.constant(0.0, shape=(1, 1))


In [15]:
x_train_age = np.array(x_train_age)
x_test_age = np.array(x_test_age)
x_train_age = x_train_age.reshape(x_train_age.shape[0], 224, 224, 1)
x_test_age = x_test_age.reshape(x_test_age.shape[0], 224, 224, 1)

        
batch_size = 64
   
train_datagen = ImageDataGenerator(rescale=1.0/255., width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, rotation_range=40,shear_range=0.2, zoom_range=0.2,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1.0/255., width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True, rotation_range=40,shear_range=0.2, zoom_range=0.2,fill_mode='nearest')
   
train_generator = train_datagen.flow(x_train_age, y_train_age, batch_size=batch_size)
test_generator = test_datagen.flow(x_test_age, y_test_age, batch_size=batch_size)

history = age_estimator_model.fit(
    train_generator, 
    epochs=40,  
    batch_size=64,  
    validation_data=test_generator)

Epoch 1/40


In [ ]:
fig = px.line(
    history.history, y=['loss', 'val_loss','accuracy'],
    labels={'index': 'epoch', 'value': 'loss'}, 
    title='Training History')
fig.show()

In [ ]:
# Guardar el modelo completo
#age_estimator_model.save('age_estimator_model.h5')
for epoch in range(40):
    age_estimator_model.save(f'model_epoch_{epoch}.h5')


